In [4]:
import requests #requests는 url을 통해 불러 오기 위해 사용

In [14]:
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
params ={'serviceKey' : 'q+LmC5LyVljCscdUobqB7LjX4lt+PwuTwejcQGR+mgW0lJYVQPIe1v7kepGi8T1i/EaTH0gMyvi7WheDbzEXvA==', 
         'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'XML', 'dataCd' : 'ASOS', 'dateCd' : 'HR',
         'startDt' : '20120101', 'startHh' : '01', 'endDt' : '20121231', 'endHh' : '01', 'stnIds' : '108' }

response = requests.get(url, params=params)
#response.content

In [15]:
content=response.text

In [16]:
### xml을 DataFrame으로 변환하기 ###
#from os import name
import xml.etree.ElementTree as et
import pandas as pd #pandas는 표형태로 쉽게 자료를 가공하기 위해 사용
import bs4 #Beautifulsoup는 태그 안의 텍스트를 가져오기 위해 사용
from lxml import html #lxml은 xml을 파싱하기 위해 beautifulsoup의 엔진으로 사용
from urllib.parse import urlencode, quote_plus, unquote

#bs4 사용하여 item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')

# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

#xml값 DataFrame으로 만들기
weather = pd.DataFrame(row_list, columns=name_list)
weather.head()

tm rnum stnId stnNm    ta taQcflg rn rnQcflg   ws wsQcflg  \
0  2012-01-01 01:00    1   108    서울   0.3       0             2.3       0   
1  2012-01-01 02:00    2   108    서울  -0.1       0             1.3       0   
2  2012-01-01 03:00    3   108    서울  -0.5       0             0.8       0   
3  2012-01-01 04:00    4   108    서울  -1.2       0             1.5       0   
4  2012-01-01 05:00    5   108    서울  -1.1       0             1.7       0   

   ... lcsCh   vs gndSttCd dmstMtphNo    ts tsQcflg m005Te m01Te m02Te m03Te  
0  ...                             40  -0.6       0   -0.3  -0.3  -0.3   0.1  
1  ...                             40  -1.2       0   -0.3  -0.3  -0.3   0.2  
2  ...    10  400        4         40  -1.7       0   -0.5  -0.4  -0.3   0.1  
3  ...    10  400                  40  -2.7       0   -0.7  -0.4  -0.3   0.2  
4  ...    10  400                  40  -2.4       0   -0.9  -0.5  -0.3   0.2  

[5 rows x 38 columns]

In [ ]:
#DataFrame CSV 파일로 저장
weather.to_csv('weather_10.csv', encoding='utf-8')